In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import col, udf
from pyspark.sql import DataFrame, Column
from pyspark.sql.window import Window
import pyspark.sql.functions as F
import pyspark.sql.types as T
from datetime import datetime, timedelta

cols_rename = [
    "filial", "cod_prod", 
    "periodo", "etiqueta", 
    "perc_dsc_cupom", "venda", 
    "venda_desconto"
]

cols_cosmos = [
    "MVVC_CD_FILIAL_MOV",
    "MVVP_NR_PRD",
    "MVVC_DT_MOV",
    "NUMERO_AUTORIZ_PAGUEMENOS",
    "MVVP_PR_DSC_ITE",
    "MVVP_VL_PRE_VDA",
    "MVVP_VL_PRD_VEN",
]

cols_pre_venda = [
    "VC_CD_FILIAL",
    "VD_CD_PRODUTO",
    "VC_DH_VENDA",
    "VD_COD_ETIQUETA_ULCH",
    "VD_PERC_DESCONTO",
    "VD_VL_PRODUTO",
    "VD_VL_PRODUTO_COM_DESCONTO",
]

cols_autorizador = [
    "ulch_sq_autorizacao",
    "ulch_preco_venda",
    "ulch_percentual_desconto",
    "ulch_fl_tipo_produto",
    "ulch_cd_barras",
    "ulch_fl_situacao",
    "ulch_sq_produto"
]

cols_produto = [
    "ulch_sq_produto",
    "xxxx_dh_cad",
    "ulch_lote",
    "ulch_dt_vencimento",
    "ulch_sq_produto"
]

In [0]:
def table_exists(
    catalog: str, 
    schema: str, 
    tablename: str
) -> bool:
    
    return spark.catalog.tableExists(f"{catalog}.{schema}.{tablename}")


def etiqueta(colname: str) -> Column:
    return F.lpad(F.trim(colname), 30, "0").cast(T.StringType())


def list_files(
    path: str, 
    start: datetime, 
    end: datetime
):
    days = (end - start).days + 1
    for day in range(days):
        dt = start + timedelta(day)
        yield f"/Volumes/raw/super_desconto/{path}/{dt:%Y/%m/%d}.parquet"

In [0]:
def view_pre_venda(
    path: str, 
    start: datetime, 
    end: datetime, 
    columns: list[str]
) -> DataFrame:
    
    col_etiqueta = columns[3]
    files_path = list(list_files(path, start, end))

    return (
        spark.read.parquet(*files_path)
        .select(columns)
        .withColumn(col_etiqueta, etiqueta(col_etiqueta))
        .withColumnsRenamed(dict(zip(columns, cols_rename)))
    )


def view_cupom(start: datetime, end: datetime) -> DataFrame:
    windows = (
        Window.partitionBy("etiqueta")
         .orderBy(col("venda_desconto").desc())
    )

    return (
        view_pre_venda("COSMOSMOV", start, end, cols_cosmos)
        .union(view_pre_venda("PRE_VENDA", start, end, cols_pre_venda))
        .withColumn("id", F.row_number().over(windows))
        .filter(col("id") == 1)
        .drop("id")
    )


def view_autorizador() -> DataFrame:
    volume = "/Volumes/raw/super_desconto/cosmos_v14b"
    file = "cosmos_v14b_dbo_ultima_chance_autorizacao.parquet"

    return (
        spark.read.parquet(f"{volume}/{file}")
        .select(cols_autorizador)
        .filter(col("ulch_fl_situacao") == "F")
        .withColumn("ulch_cd_barras", etiqueta("ulch_cd_barras"))
        .withColumn("ulch_percentual_desconto", F.coalesce("ulch_percentual_desconto", F.lit(0)))
    )


def view_produto() -> DataFrame:
    volume = "/Volumes/raw/super_desconto/cosmos_v14b"
    file = "cosmos_v14b_dbo_ultima_chance_produto.parquet"

    return (
        spark.read.parquet(f"{volume}/{file}")
        .select(cols_produto)
        .withColumn("ulch_lote", F.upper(F.trim("ulch_lote")))
    )

In [0]:
# consulta full load
cupom = view_cupom(datetime(2023, 1, 1), datetime(2023, 3, 31))
autorizador = view_autorizador()
produto = view_produto()

view_create = (
    cupom
     .join(autorizador, cupom.etiqueta == autorizador.ulch_cd_barras)
     .join(produto, autorizador.ulch_sq_produto == produto.ulch_sq_produto)
     .select(
        autorizador.ulch_sq_autorizacao,
        produto.ulch_sq_produto,
        produto.xxxx_dh_cad,
        cupom.periodo.alias("dt_venda"),
        cupom.filial,
        cupom.cod_prod,
        produto.ulch_lote,
        produto.ulch_dt_vencimento,
        cupom.etiqueta,
        cupom.perc_dsc_cupom,
        cupom.venda,
        cupom.venda_desconto,
        autorizador.ulch_preco_venda.alias("ulch_preco_venda"), 
        autorizador.ulch_percentual_desconto, 
        autorizador.ulch_fl_tipo_produto
     )
)

In [0]:
# full table
view_create.coalesce(1).write.mode("overwrite").format("delta").saveAsTable("bronze.super_desconto.venda")

In [0]:
%sql

select count(*) from bronze.super_desconto.venda